In [38]:
import json
with open("../config/parametros.json", "r") as file:
    data = json.load(file)
    print(data)

{'ano_inicio_scraping': '1970'}


In [39]:
print(data['ano_inicio_scraping'])

1970


In [43]:
for i in range(1, 5+1):
    print(i)

1
2
3
4
5


In [46]:
produto = "processamento"
dicionario_subproduto = {
        "Viníferas":"1"
        , "Americanas e híbridas":"2"
        , "Uvas de mesa":"3"
        , "Sem classificação":"4"
        }
for subproduto in dicionario_subproduto.items():
    print(subproduto[0])

Viníferas
Americanas e híbridas
Uvas de mesa
Sem classificação
